In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
import time
import openpyxl
from selenium.webdriver.common.keys import Keys
import pandas as pd
from bs4 import BeautifulSoup

# 최종 영화 csv 파일로 사용 / id number 있으면 그것도 불러옴
mlist = pd.read_csv('./결측치 제외.csv')
searching = mlist['영화명']

try:
    wb = openpyxl.load_workbook("KOBIS.xlsx")
    sheet = wb.active
    print("불러오기 완료")
except:
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["검색어", "영화명", "요약정보", "개봉일", "재개봉일", "재개봉여부", "스크린수", "상영횟수", "누적매출액", "누적관객수"])
    print("새로운 파일을 만들었습니다.")

driver = webdriver.Chrome("./chromedriver")

for i in range(len(mlist)):
    try:
        # 영화 검색
        driver.get("http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do")
        time.sleep(2)

        search_box = driver.find_element_by_css_selector("input#inp_solrSearch")
        search_box.send_keys(searching[i])
        search_button = driver.find_element_by_css_selector("button.btn_sch")
        search_button.click()

        time.sleep(3)

        # 검색 결과의 첫 번째 영화 클릭
        mname_button = driver.find_element_by_xpath("//a[@href='#none']")
        mname_button.click()
        time.sleep(2)

        # 검색한 영화와 실제 결과로 나온 영화가 일치하는지 알아보기 위해 수집
        movie_name = driver.find_element_by_css_selector("div.hd_layer strong.tit").text.strip()

        # 상세정보에서 개봉일/재개봉일 수집
        summary_info = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[3].text.strip()
        open_date = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[4].text.strip()

        try:
            reopen = driver.find_element_by_xpath("//dt[text()='재개봉일']")
            reopen_date = driver.find_elements_by_css_selector("dl.ovf.cont > dd")[7].text.strip()
            re_whether = "1"
        except:
            reopen_date = ""
            re_whether = "0"

        # 통계정보로 넘어감
        stat_button = driver.find_element_by_xpath("//a[text()='통계정보']")
        stat_button.click()
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 재개봉 영화의 경우 공식 통계 숫자가 재개봉 영화의 통계를 포함하고 있어 정확하지 x, 전체 누적매출액이 아님
        # 스크린수, 상영횟수, 누적매출액, 누적관객수 구하기 - 상영타입별 누적통계에서
        try:
            num_list = soup.select("tr.summary > td.tar")
            total_screen = num_list[3].text.strip()
            total_show = num_list[4].text.strip()
            total_revenue = num_list[5].text.strip()
            total_attendance = num_list[6].text.strip()
        # 상영타입별 누적통계 없을 경우 - 보통 오래된 영화가 대부분. 일단 서울시 공식통계 수집
        except:
            num_list = soup.select(".cont_tab > div:nth-of-type(2) > div:nth-of-type(1) > table > tbody > tr > td[class=tar]")
            total_screen = "서울" + num_list[0].text.strip()
            total_show = ""
            total_revenue = "서울" + num_list[1].text.strip()
            total_attendance = "서울" + num_list[2].text.strip()

        print(searching[i], "++", movie_name, "요약정보:", summary_info, "개봉:", open_date, "재개봉:", reopen_date, re_whether,
              "스크린수:", total_screen, "상영횟수:", total_show, "누적매출액:", total_revenue, "누적관객수:", total_attendance)
        print("="*50)
        sheet.append([searching[i], movie_name, summary_info, open_date, reopen_date, re_whether, total_screen, total_show,
                      total_revenue, total_attendance])
        wb.save("KOBIS.xlsx")

    except:
        print(searching[i], "오류 발생")
        continue

driver.close()
wb.save("KOBIS.xlsx")

불러오기 완료
